In [69]:
# import libraries
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, cross_val_score
import statsmodels.api as sm

In [70]:
# import feature data
data = pd.read_csv('../data/processed_data/Wildfire_Weather_2020_2024.csv')

In [71]:
# view feature data
data.head()

,unique_id,size (km2),perimiter (km),startdateday,startdatemonth,startdateyear,startdateseason,enddateday,enddatemonth,enddateyear,...,d2m,t2m,msl,sp,lai_hv,lai_lv,tp,ssr,datetime,dist
0,2020_22784,4.93,11.11,6,1,2020,Winter,13,1,2020,...,278.64328,284.38715,102901.520,102447.460,2.988195,2.407728,0.000021,8017088.0,2020-01-09,6565.323606
1,2020_22784,4.93,11.11,6,1,2020,Winter,13,1,2020,...,287.59494,292.01648,102516.750,102070.190,2.986613,2.406662,0.000020,5444096.0,2020-01-10,6565.323606
2,2020_22784,4.93,11.11,6,1,2020,Winter,13,1,2020,...,287.74893,289.71127,102244.630,101797.350,2.983510,2.404584,0.003486,8655968.0,2020-01-12,6565.323606
3,2020_22784,4.93,11.11,6,1,2020,Winter,13,1,2020,...,278.82760,283.12076,102448.375,101995.414,2.992889,2.410894,0.000000,12236688.0,2020-01-06,6565.323606
4,2020_22784,4.93,11.11,6,1,2020,Winter,13,1,2020,...,292.18143,294.35640,101801.750,101360.100,2.985026,2.405601,0.025873,2535104.0,2020-01-11,6565.323606


In [72]:
# view feature data
data.describe()

,size (km2),perimiter (km),startdateday,startdatemonth,startdateyear,enddateday,enddatemonth,enddateyear,duration,fire_line (km),...,v10,d2m,t2m,msl,sp,lai_hv,lai_lv,tp,ssr,dist
count,32082.000000,32082.000000,32082.00000,32082.000000,32082.000000,32082.000000,32082.000000,32082.000000,32082.000000,32082.000000,...,32082.000000,32082.000000,32082.000000,32082.000000,32082.000000,32082.000000,32082.000000,32082.000000,3.208200e+04,32082.000000
mean,19.627479,29.133438,9.29088,5.795742,2021.479677,21.232654,5.795742,2021.479677,12.941774,2.496167,...,0.341190,280.752792,290.498979,101587.466259,95404.544994,2.717593,1.764379,0.001590,1.680758e+07,9411.910052
std,49.696907,27.715941,6.34828,2.779752,1.122430,6.779502,2.779752,1.122430,5.844891,3.886487,...,2.349705,8.882896,7.656374,614.691170,7389.700655,1.142001,0.716741,0.004740,5.577529e+06,3635.845120
min,4.070000,9.260000,1.00000,1.000000,2020.000000,1.000000,1.000000,2020.000000,1.000000,0.350000,...,-10.041690,247.640150,256.153400,99178.340000,70101.730000,0.000000,0.000000,0.000000,4.577280e+05,250.207361
25%,5.360000,15.740000,4.00000,3.000000,2020.000000,16.000000,3.000000,2020.000000,8.000000,0.930000,...,-1.024666,274.147685,285.338530,101231.480000,91950.432500,2.030273,1.265003,0.000000,1.331969e+07,6686.224160
50%,7.930000,20.370000,8.00000,6.000000,2022.000000,22.000000,6.000000,2022.000000,12.000000,1.470000,...,0.299106,280.678035,290.937680,101582.675000,97986.463000,2.593112,1.686295,0.000029,1.739407e+07,9681.550126
75%,14.150000,31.480000,14.00000,9.000000,2022.000000,27.000000,9.000000,2022.000000,17.000000,2.620000,...,1.707381,287.887285,296.530053,101939.154250,100956.050000,3.549260,2.393430,0.000806,2.063463e+07,12217.278808
max,806.240000,298.170000,31.00000,11.000000,2024.000000,31.000000,11.000000,2024.000000,31.000000,140.600000,...,10.673229,299.197050,311.581670,104212.430000,103518.164000,5.127843,3.819295,0.142104,2.985043e+07,17855.839385


In [73]:
# view feature data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32082 entries, 0 to 32081
Data columns (total 34 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   unique_id              32082 non-null  object 
 1   size (km2)             32082 non-null  float64
 2   perimiter (km)         32082 non-null  float64
 3   startdateday           32082 non-null  int64  
 4   startdatemonth         32082 non-null  int64  
 5   startdateyear          32082 non-null  int64  
 6   startdateseason        32082 non-null  object 
 7   enddateday             32082 non-null  int64  
 8   enddatemonth           32082 non-null  int64  
 9   enddateyear            32082 non-null  int64  
 10  enddatesesason         32082 non-null  object 
 11  duration               32082 non-null  int64  
 12  fire_line (km)         32082 non-null  float64
 13  fire_spread (km2/day)  32082 non-null  float64
 14  fire_speed (km/day)    32082 non-null  float64
 15  do

In [74]:
# split data
train_data = data[data['startdateyear'].isin([2020, 2021, 2022])]
test_data = data[data['startdateyear'].isin([2023, 2024])]

In [75]:
# Predictors and Targets
predictors = ['startdateseason', 'u10', 'v10', 'd2m', 't2m', 'msl', 'sp', 'lai_hv', 'lai_lv', 'tp', 'ssr']
target1 = 'size (km2)'
target2 = 'fire_spread (km2/day)'
target3 = 'duration'

In [76]:
# Split the data
X_train = train_data[predictors]
y1_train = train_data[target1]
y2_train = train_data[target2]
y3_train = train_data[target3]


X_test = test_data[predictors]
y1_test = test_data[target1]
y2_test = test_data[target2]
y3_test = test_data[target3]

In [77]:
# --- Step 4: One-hot encode the 'startdateseason' variable ---
X_train = pd.get_dummies(X_train, drop_first=True).astype(float)
X_test = pd.get_dummies(X_test, drop_first=True).astype(float)

# Ensure test set has same columns as train (fill missing with 0)
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

In [78]:
# Set seed and CV strategy
random_seed = 1
kf = KFold(n_splits=5, shuffle=True, random_state=random_seed)

In [79]:
# Model 1: Predict size (km2)
# intialize the model
model1 = LinearRegression()

In [80]:
# Cross-validation: MSE and RMSE
neg_mse1 = cross_val_score(model1, X_train, y1_train, cv=kf, scoring='neg_mean_squared_error')
rmse1 = np.sqrt(-neg_mse1)

In [81]:
# Cross-validation: R²
r2_scores1 = cross_val_score(model1, X_train, y1_train, cv=kf, scoring='r2')

In [82]:
# Statsmodels for coeffs/p-values and R² on full training data
X1_const = sm.add_constant(X_train)
ols1 = sm.OLS(y1_train, X1_const).fit()

print("\nCoefficients and p-values of the full model for predicting size (km2):")
print(ols1.summary())

print("\nEvaluation metrics for full model for predicting size (km2):")
print(f"\nR² (full training data): {ols1.rsquared:.4f}")
print(f"Correlation coefficient (r): {ols1.rsquared**0.5:.4f}")

print(f"5-fold CV Mean R²: {r2_scores1.mean():.4f}")
print(f"5-fold CV Mean MSE: {-neg_mse1.mean():.4f}")
print(f"5-fold CV Mean RMSE: {rmse1.mean():.4f}")



Coefficients and p-values of the full model for predicting size (km2):
                            OLS Regression Results                            
Dep. Variable:             size (km2)   R-squared:                       0.100
Model:                            OLS   Adj. R-squared:                  0.100
Method:                 Least Squares   F-statistic:                     221.6
Date:                Fri, 04 Jul 2025   Prob (F-statistic):               0.00
Time:                        13:03:56   Log-Likelihood:            -1.3824e+05
No. Observations:               25917   AIC:                         2.765e+05
Df Residuals:                   25903   BIC:                         2.766e+05
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------

In [83]:
# Model 2: Predict fire_spread (km2/day)
model2 = LinearRegression()

In [84]:
# Cross-validation: MSE and RMSE
neg_mse2 = cross_val_score(model2, X_train, y2_train, cv=kf, scoring='neg_mean_squared_error')
rmse2 = np.sqrt(-neg_mse2)

In [85]:
# Cross-validation: R²
r2_scores2 = cross_val_score(model2, X_train, y2_train, cv=kf, scoring='r2')

In [86]:
# Statsmodels summary for second model (full training data)
X2_const = sm.add_constant(X_train)
ols2 = sm.OLS(y2_train, X2_const).fit()

print("\nCoefficients and p-values of the full model for predicting fire spread (km2 / day):")
print(ols2.summary())

print("\nEvaluation metrics for full model for predicting fire spread (km2 / day):")
print(f"\nR² (full training data): {ols2.rsquared:.4f}")
print(f"Correlation coefficient (r): {ols2.rsquared**0.5:.4f}")

print(f"5-fold CV Mean R²: {r2_scores2.mean():.4f}")
print(f"5-fold CV Mean MSE: {-neg_mse2.mean():.4f}")
print(f"5-fold CV Mean RMSE: {rmse2.mean():.4f}")


Coefficients and p-values of the full model for predicting fire spread (km2 / day):
                              OLS Regression Results                             
Dep. Variable:     fire_spread (km2/day)   R-squared:                       0.126
Model:                               OLS   Adj. R-squared:                  0.126
Method:                    Least Squares   F-statistic:                     287.7
Date:                   Fri, 04 Jul 2025   Prob (F-statistic):               0.00
Time:                           13:03:56   Log-Likelihood:                -64785.
No. Observations:                  25917   AIC:                         1.296e+05
Df Residuals:                      25903   BIC:                         1.297e+05
Df Model:                             13                                         
Covariance Type:               nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
----

In [87]:
# Model 3: Predict duration (days)
model3 = LinearRegression()

In [88]:
# Cross-validation: MSE and RMSE
neg_mse3 = cross_val_score(model3, X_train, y3_train, cv=kf, scoring='neg_mean_squared_error')
rmse3 = np.sqrt(-neg_mse3)

In [89]:
# Cross-validation: R²
r2_scores3 = cross_val_score(model3, X_train, y3_train, cv=kf, scoring='r2')

In [90]:
# Statsmodels summary for third model (full training data)
X3_const = sm.add_constant(X_train)
ols3 = sm.OLS(y3_train, X3_const).fit()

print("\nCoefficients and p-values of the full model for predicting duration (days):")
print(ols3.summary())

print("\nEvaluation metrics for full model for predicting duration (days):")
print(f"\nR² (full training data): {ols3.rsquared:.4f}")
print(f"Correlation coefficient (r): {ols3.rsquared**0.5:.4f}")

print(f"5-fold CV Mean R²: {r2_scores3.mean():.4f}")
print(f"5-fold CV Mean MSE: {-neg_mse3.mean():.4f}")
print(f"5-fold CV Mean RMSE: {rmse3.mean():.4f}")


Coefficients and p-values of the full model for predicting duration (days):
                            OLS Regression Results                            
Dep. Variable:               duration   R-squared:                       0.134
Model:                            OLS   Adj. R-squared:                  0.134
Method:                 Least Squares   F-statistic:                     308.7
Date:                Fri, 04 Jul 2025   Prob (F-statistic):               0.00
Time:                        13:03:57   Log-Likelihood:                -81099.
No. Observations:               25917   AIC:                         1.622e+05
Df Residuals:                   25903   BIC:                         1.623e+05
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------

In [91]:
def iterative_feature_elimination(X, y, alpha=0.05):
    X_const = sm.add_constant(X)
    ols = sm.OLS(y, X_const).fit()

    while True:
        pvalues = ols.pvalues.drop('const')
        max_p = pvalues.max()
        if max_p <= alpha:
            break  # All predictors significant enough

        # Remove predictor with max p-value
        worst_feature = pvalues.idxmax()
        print(f"Removing '{worst_feature}' with p-value {max_p:.4f}")
        X = X.drop(columns=[worst_feature])

        X_const = sm.add_constant(X)
        ols = sm.OLS(y, X_const).fit()

    return X, ols

# Set random seed and CV strategy (reuse kf)
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# For Model 1
print("\n--- Iterative elimination of the model for predicting size (km2) ---")
X1_reduced, ols1_reduced = iterative_feature_elimination(X_train, y1_train)

model1_reduced = LinearRegression()
neg_mse1_reduced = cross_val_score(model1_reduced, X1_reduced, y1_train, cv=kf, scoring='neg_mean_squared_error')
rmse1_reduced = np.sqrt(-neg_mse1_reduced)
r2_scores1_reduced = cross_val_score(model1_reduced, X1_reduced, y1_train, cv=kf, scoring='r2')

print(f"\nReduced model for predicting size (km2) predictors: {list(X1_reduced.columns)}")
print("\nCoefficients and p-values after elimination (model for predicting size (km2)):")
print(ols1_reduced.summary())
print("\nEvaluation metrics for reduced model for predicting size (km2):")
print(f"R² (full training data): {ols1_reduced.rsquared:.4f}")
print(f"Correlation coefficient (r): {ols1_reduced.rsquared**0.5:.4f}")
print(f"5-fold CV Mean R²: {r2_scores1_reduced.mean():.4f}")
print(f"5-fold CV Mean MSE: {-neg_mse1_reduced.mean():.4f}")
print(f"5-fold CV Mean RMSE: {rmse1_reduced.mean():.4f}")

# For Model 2
print("\n--- Iterative elimination of model for predicting fire spread (km2 / day) ---")
X2_reduced, ols2_reduced = iterative_feature_elimination(X_train, y2_train)

model2_reduced = LinearRegression()
neg_mse2_reduced = cross_val_score(model2_reduced, X2_reduced, y2_train, cv=kf, scoring='neg_mean_squared_error')
rmse2_reduced = np.sqrt(-neg_mse2_reduced)
r2_scores2_reduced = cross_val_score(model2_reduced, X2_reduced, y2_train, cv=kf, scoring='r2')

print(f"\nReduced Model 2 predictors: {list(X2_reduced.columns)}")
print("\nCoefficients and p-values for reduced model for predicting fire spread (km2 / day):")
print(ols2_reduced.summary())
print("\nEvaluation metrics for reduced model for predicting fire spread (km2 / day):")
print(f"R² (full training data): {ols2_reduced.rsquared:.4f}")
print(f"Correlation coefficient (r): {ols2_reduced.rsquared**0.5:.4f}")
print(f"5-fold CV Mean R²: {r2_scores2_reduced.mean():.4f}")
print(f"5-fold CV Mean MSE: {-neg_mse2_reduced.mean():.4f}")
print(f"5-fold CV Mean RMSE: {rmse2_reduced.mean():.4f}")

# For Model 3
print("\n--- Iterative elimination of model for duration (days) ---")
X3_reduced, ols3_reduced = iterative_feature_elimination(X_train, y3_train)

model3_reduced = LinearRegression()
neg_mse3_reduced = cross_val_score(model3_reduced, X3_reduced, y3_train, cv=kf, scoring='neg_mean_squared_error')
rmse3_reduced = np.sqrt(-neg_mse3_reduced)
r2_scores3_reduced = cross_val_score(model3_reduced, X3_reduced, y3_train, cv=kf, scoring='r2')

print(f"\nReduced Model 3 predictors: {list(X3_reduced.columns)}")
print("\nCoefficients and p-values for reduced model for duration (days):")
print(ols3_reduced.summary())
print("\nEvaluation metrics for reduced model for duration (days):")
print(f"R² (full training data): {ols3_reduced.rsquared:.4f}")
print(f"Correlation coefficient (r): {ols3_reduced.rsquared**0.5:.4f}")
print(f"5-fold CV Mean R²: {r2_scores3_reduced.mean():.4f}")
print(f"5-fold CV Mean MSE: {-neg_mse3_reduced.mean():.4f}")
print(f"5-fold CV Mean RMSE: {rmse3_reduced.mean():.4f}")



--- Iterative elimination of the model for predicting size (km2) ---
Removing 'v10' with p-value 0.7278
Removing 'sp' with p-value 0.6437
Removing 'u10' with p-value 0.5102
Removing 'msl' with p-value 0.2131

Reduced model for predicting size (km2) predictors: ['d2m', 't2m', 'lai_hv', 'lai_lv', 'tp', 'ssr', 'startdateseason_Spring', 'startdateseason_Summer', 'startdateseason_Winter']

Coefficients and p-values after elimination (model for predicting size (km2)):
                            OLS Regression Results                            
Dep. Variable:             size (km2)   R-squared:                       0.100
Model:                            OLS   Adj. R-squared:                  0.100
Method:                 Least Squares   F-statistic:                     319.8
Date:                Fri, 04 Jul 2025   Prob (F-statistic):               0.00
Time:                        13:03:57   Log-Likelihood:            -1.3824e+05
No. Observations:               25917   AIC:              